<a href="https://colab.research.google.com/github/alexsimas07/Modelo_Preditivo_Deteccao_fraude_financeira/blob/main/Deteccao_Fraude_financeira.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Modelo de detecção de fraude para dados financeiros

Sobre nosso dataset:

* Step = mapeia uma unidade de tempo no mundo real. Neste caso, 1 passo é 1 hora de tempo. Total de etpas 744 (simulação de 30 dias).
* Type = CASH-IN, CASH-OUT, DEBIT, PAYMENT and TRANSFER.
* amount = valor da transação em moeda local.
* nameOrig = Cliente que iniciou a transação
* oldbalanceOrig = saldo inicial antes da transação
* newbalanceOrig = novo saldo após a transação
* nameDest = cliente que é o destinatário da transação
* oldbalanceDest = destinatário do saldo inicial antes da transação
* newbalanceDest = Novo destinatário do saldo após a transação.
* isFraud = São as transações feitas pelos agentes fraudulentos dentro da simulação. Neste conjunto de dados especifico, o comportamento fraudulento dos agentes visa lucar ao assumir o controle das contas dos clientes e tentar esvaziar os fundos transferidos para outra conta e depois sacando do sistema.
* isFlaggedFraud = O modelo de negócio visa controlar transferências massivas de uma conta para outra e sinaliza tentativas ilegais. Uma tentativa ilegal neste conjunto de dados é uma tentativa de transferir mais de 200.000 em uma única transação.



In [ ]:
# importando as bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv("/content/fraud_dataset_example.csv")
df.head()

In [ ]:
df.columns

In [ ]:
df = df[['isFraud','isFlaggedFraud','step', 'type', 'amount', 'nameOrig', 'oldbalanceOrg', 'newbalanceOrig',
       'nameDest', 'oldbalanceDest', 'newbalanceDest', ]]
df.head()

In [ ]:
# Renomeando as colunas
colunas = {
    'isFraud':'fraude',
    'isFlaggedFraud':'possivel_fraude',
    'step':'tempo',
    'type':'tipo',
    'amount':'valor',
    'nameOrig':'cliente1',
    'oldbalanceOrg':'saldo_inicial_c1',
    'newbalanceOrig':'novo_saldo_c1',
    'nameDest':'cliente2',
    'oldbalanceDest':'saldo_inicial_c2',
    'newbalanceDest':'novo_saldo_c2'
}

In [ ]:
df = df.rename(columns=colunas)
df.head()

In [ ]:
df.describe()

In [ ]:
df.describe().T

### Verificando valores nulos

In [ ]:
df.isnull().sum()

### Encoding

In [ ]:
# Instalando o pandas profiling
!pip install ydata-profiling

In [ ]:
from ydata_profiling import ProfileReport

ProfileReport(df)

In [ ]:
df_dumm = pd.get_dummies(pd.Series(list(df['tipo'])),dtype=float)
df_dumm.head()

In [ ]:
df_dumm_conv = ['CASH_IN', 'CASH_OUT', 'DEBIT', 'PAYMENT', 'TRANSFER']
df_dumm[df_dumm_conv] = df_dumm[df_dumm_conv].astype(float)
df_dumm.head()

In [ ]:
df = pd.merge(df,df_dumm,left_index=True,right_index=True)
df.head()

Tive que fazer essa manobra pra poder juntar dois dataset e converter os valores das colunas CASH_IN, CASH_OUT, DEBIT, PAYMENT e TRANSFER.

In [ ]:
df = df.drop(['cliente1', 'cliente2', 'possivel_fraude'], axis=1)
df.head()

In [ ]:
df=df.drop(['tipo'], axis=1)
df.head()

### Regressão Logística

In [ ]:
x = df.drop('fraude', axis=1)
y = df['fraude']

In [ ]:
from sklearn.model_selection import train_test_split

SEED = 42

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=SEED)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

lr = LogisticRegression(max_iter = 1000, random_state=SEED)
lr.fit(x_train, y_train)
y_pred = lr.predict(x_test)

print('Acurácia: ', metrics.accuracy_score(y_test, y_pred))
print('Precisão: ', metrics.precision_score(y_test, y_pred))
print('Recall: ', metrics.recall_score(y_test, y_pred))
print('F1: ', metrics.f1_score(y_test, y_pred))

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.datasets import make_classification
from sklearn.metrics import ConfusionMatrixDisplay
%matplotlib inline

cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()

In [ ]:
y_pred_proba = lr.predict_proba(x_test)[::,1]
fpr, tpr, _ = metrics.roc_curve(y_test, y_pred_proba)
auc = metrics.roc_auc_score(y_test, y_pred_proba)

plt.rcParams['figure.figsize'] = (12,8)
plt.plot(fpr, tpr, label="LR, auc="+str(auc))
plt.plot([0,1], [0,1], color = 'red', lw=2, linestyle='--')
plt.legend(loc=4)

Aqui enxergamos que nossas classes estão desbalanceadas, neste caso precisamos balancear as classes para não deixar o modelo tendencioso.

* UNDERSAMPLING = reduzir dados de não fraude para 116 (errado)
* OVERSAMPLING = copiar os dados da classe minoritária até chegar no valor da classe majoritária,mas tem problemas por exemplo quando aplicamos a técnica teoricamente colocamos valores sinteticos para treinar nosso modelo, ou seja, não são valores reais.

SMOTE = Algoritmo KNN (aproximação dos vizinhos)


### Balanceamento de dados

In [ ]:
!pip install -U imbalanced-learn

In [ ]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=SEED)

In [ ]:
x_resampled, y_resampled = smote.fit_resample(x, y)

In [ ]:
df_balanced = pd.concat([x_resampled, y_resampled], axis=1)
df_balanced.head()

In [ ]:
df_balanced.groupby('fraude').count()

In [ ]:
df_balanced.describe()

In [ ]:
df_balanced

### Novo teste de Regressão Logistica

In [ ]:
x = df_balanced.drop('fraude', axis=1)
y = df_balanced['fraude']

In [ ]:
from sklearn.model_selection import train_test_split

SEED = 42

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=SEED)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

lr = LogisticRegression(solver='lbfgs',class_weight='balanced', max_iter=10000)

lr.fit(x_train, y_train)
y_pred=lr.predict(x_test)

print("Acurácia:", metrics.accuracy_score(y_test, y_pred))
print("Precisão:", metrics.precision_score(y_test, y_pred))
print("Recall:", metrics.recall_score(y_test, y_pred))
print("F1:", metrics.f1_score(y_test, y_pred))

In [ ]:
cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()

In [ ]:
y_pred_proba = lr.predict_proba(x_test)[::,1]
fpr, tpr, _ = metrics.roc_curve(y_test, y_pred_proba)
auc = metrics.roc_auc_score(y_test, y_pred_proba)

plt.rcParams['figure.figsize'] = (12,8)
plt.plot(fpr, tpr, label="LR, auc="+str(auc))
plt.plot([0,1], [0,1], color = 'red', lw=2, linestyle='--')
plt.legend(loc=4)

### Formulando as hipóteses

In [ ]:
ProfileReport(df_balanced)

** Inserir as hipóteses**
- pegando como paramentro a coluna fraude, a correlação mais alta está com CASH_IN, CASHOUT, PAYMENT, tempo e TRANSFER
- As colunas novo saldoc1 e c2 tem baixa correlação e também saldo inicial c1 e c2, logo esses valores não são importantes para a fraude ocorrer.

### Árvore de decisão

In [ ]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(max_depth=5, random_state=SEED)

In [ ]:
model = dt.fit(x_train, y_train)
y_pred = dt.predict(x_test)

print("Acurácia:", metrics.accuracy_score(y_test, y_pred))
print("Precisão:", metrics.precision_score(y_test, y_pred))
print("Recall:", metrics.recall_score(y_test, y_pred))
print("F1:", metrics.f1_score(y_test, y_pred))

In [ ]:
cm_decision = confusion_matrix(y_test, y_pred)
disp_decision = ConfusionMatrixDisplay(confusion_matrix=cm_decision)
disp_decision.plot()

In [ ]:
y_pred_proba = dt.predict_proba(x_test)[::,1]
fpr, tpr, _ = metrics.roc_curve(y_test, y_pred_proba)
auc = metrics.roc_auc_score(y_test, y_pred_proba)

plt.rcParams['figure.figsize'] = (12,8)
plt.plot(fpr, tpr, label="dt, auc="+str(auc))
plt.plot([0,1], [0,1], color = 'red', lw=2, linestyle='--')
plt.legend(loc=4)

### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(max_depth=5, random_state=SEED, n_estimators=120)

In [ ]:
model = rf.fit(x_train, y_train)
y_pred = rf.predict(x_test)

print("Acurácia:", metrics.accuracy_score(y_test, y_pred))
print("Precisão:", metrics.precision_score(y_test, y_pred))
print("Recall:", metrics.recall_score(y_test, y_pred))
print("F1:", metrics.f1_score(y_test, y_pred))

In [ ]:
cm_random = confusion_matrix(y_test, y_pred)
disp_decision = ConfusionMatrixDisplay(confusion_matrix=cm_random)
disp_decision.plot()

In [ ]:
y_pred_proba = rf.predict_proba(x_test)[::,1]
fpr, tpr, _ = metrics.roc_curve(y_test, y_pred_proba)
auc = metrics.roc_auc_score(y_test, y_pred_proba)

plt.rcParams['figure.figsize'] = (12,8)
plt.plot(fpr, tpr, label="rf, auc="+str(auc))
plt.plot([0,1], [0,1], color = 'red', lw=2, linestyle='--')
plt.legend(loc=4)

### Análise de Métricas

In [ ]:
classifiers = [
    LogisticRegression(max_iter=1000, random_state=SEED),
    DecisionTreeClassifier(max_depth=5, random_state=SEED),
    RandomForestClassifier(max_depth=5, random_state=SEED)]

In [ ]:
for clf in classifiers:
  clf.fit(x_train, y_train)
  name=clf.__class__.__name__
  print("="*30)
  print(name)
  print('****Results****')
  y_pred = clf.predict(x_test)
  print("Accuracy:", metrics.accuracy_score(y_test, y_pred))
  print("Precision:", metrics.precision_score(y_test, y_pred))
  print("Recall:", metrics.recall_score(y_test, y_pred))
  print("F1:", metrics.f1_score(y_test, y_pred))
  print("="*30)

  cm = confusion_matrix(y_test, y_pred)
  disp = ConfusionMatrixDisplay(confusion_matrix=cm)
  disp.plot()

### Melhorando o modelo Random Forest

In [ ]:
n_estimators = np.arange(20,200, step=20)
criterion = ["gini", "entropy"]
max_features = ["auto", "sqrt", "log2"]
max_depth = list(np.arange(2, 10, step=1))
min_samples_split = list(np.arange(2, 10, step=1))
min_samples_leaf = [1,2,4]
bootstrap = [True, False]

In [ ]:
param_grid = {
    'n_estimators': n_estimators,
    'criterion': criterion,
    'max_features': max_features,
    'max_depth': max_depth,
    'min_samples_split': min_samples_split,
    'min_samples_leaf': min_samples_leaf,
    'bootstrap': bootstrap
}

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

SEED = 70

rforest = RandomForestClassifier(random_state=SEED)

In [ ]:
random_cv = RandomizedSearchCV(
    rforest, param_grid, n_iter=5, cv=3, scoring="recall", n_jobs=-1, random_state=SEED
)

In [ ]:
%%time
rcv = random_cv.fit(x, y)

In [ ]:
pd.DataFrame(rcv.cv_results_)

In [ ]:
rcv.best_params_

### Resultados Finais

In [ ]:
rfn = RandomForestClassifier(**rcv.best_params_, random_state=SEED)

model = rfn.fit(x_train, y_train)
y_pred = rfn.predict(x_test)

In [ ]:
print("Acurácia:", metrics.accuracy_score(y_test, y_pred))
print("Precisão:", metrics.precision_score(y_test, y_pred))
print("Recall:", metrics.recall_score(y_test, y_pred))
print("F1:", metrics.f1_score(y_test, y_pred))

In [ ]:
cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()